# This is a tutorial to work with cell painting data
[Inspired from [pycitominer](https://github.com/cytomining/pipeline-examples/blob/main/1.profile.ipynb)]
## ⚙️ Cell Painting Data Processing Steps

### 1. Aggregation
After image analysis, **single-cell profiles** are generated from the segmented images.  
Each **well** may contain multiple images corresponding to the same **perturbation or treatment**.  
These single-cell profiles are **aggregated to the well level** (e.g., by computing the median or mean across cells).  
The resulting well-level profiles are then **re-annotated with metadata** (plate, well, compound, concentration, etc.) to create a unified dataset.

---

### 2. Normalization
To ensure consistent comparisons across plates and experiments, **feature values are normalized** relative to control wells — typically **DMSO** for chemical perturbations.  
Normalization removes plate-to-plate and systematic technical variability, allowing biological differences to be more accurately captured.

---

### 3. Feature Selection
CellProfiler extracts thousands of features, many of which can be **redundant or highly correlated**.  
To reduce noise and improve interpretability, **feature selection** is applied to retain only the most informative and non-redundant features.  
This step improves data quality and computational efficiency for downstream analysis.

---

### 4. Aggregation of Consensus Profiles
Finally, **well-level profiles** corresponding to the same **perturbation or treatment** (e.g., multiple wells, replicates, or concentrations) are **aggregated into consensus profiles**.  
These consensus profiles represent the **robust morphological signature** of each perturbation, which can be used for reproducibility assessment and biological interpretation.

---
### Levels of Cell Painting Data Processing

Cell Painting data can be represented at multiple levels of processing, from raw microscopy images to consensus-level profiles.  
Each level provides increasing abstraction and refinement of the data.

| **Data Description**                              | **Level** | **Details** |
| :------------------------------------------------ | :--------: | :----------- |
| **Images**                                        | Level 1 | Raw microscopy images captured from the Cell Painting assay. Each image corresponds to a single field of view in a specific well and channel. |
| **Single-cell profiles**                 | Level 2 | Per-cell morphological measurements extracted from images using for example *CellProfiler*. |
| **Aggregated profiles with metadata information** | Level 3 | Single-cell profiles aggregated to the **well level** (e.g., median across all cells in a well) and annotated with **metadata** such as plate, well, and treatment. |
| **Normalized aggregated profiles**                | Level 4a | Well-level profiles normalized relative to **control wells** (e.g., DMSO) to correct for plate or batch effects. |
| **Normalized and feature-selected profiles**      | Level 4b | Normalized profiles with redundant or noisy features removed to retain only the **most informative and stable features**. |
| **Consensus profiles**                            | Level 5 | Aggregation of replicate or multi-concentration profiles to create a **final consensus signature** per perturbation or treatment. Used for reproducibility assessment and biological interpretation. |

---

✅ *Summary:*  
**Level 1 → Level 5** represents the data refinement pipeline:  
**Raw images → Single-cell → Well-level → Normalized → Feature-selected → Consensus profiles**


## 📚 Glossary

**Plates**  
Containers that hold multiple wells where cells are grown and treated with perturbations or compounds.  
Cell Painting experiments commonly use **96-well** or **384-well** plates.

**Wells**  
Individual compartments within a plate. Each well contains a population of cells exposed to a specific **perturbation**, **treatment**, or **control** condition.

**Batch**  
A group of plates processed together under similar experimental conditions (e.g., same day, same microscope, same operator).  
Batch information is critical for identifying and correcting **batch effects** — systematic differences unrelated to biology.

**Profiles**  
High-dimensional representations of cell morphology obtained after feature extraction.  
Each profile summarizes thousands of **features** that describe cellular shape, texture, and intensity across multiple imaging channels.

**Features**  
Quantitative measurements extracted from microscopy images (using software like CellProfiler).  
Examples include cell size, nuclear intensity, texture patterns, and spatial relationships between organelles.  
Features are often normalized and aggregated to create a **profile** per well, treatment, or cell population.

**Metadata**  
Contextual information describing each sample, such as plate ID, well position, compound name, concentration, gene target, or imaging parameters.  
Metadata allows linking image-derived features to experimental design and biological meaning.

**Perturbation / Treatment**  
The experimental condition applied to cells to induce a phenotype of interest.  
This can be:
- A **chemical compound** (e.g., drug or small molecule). The same chemical tested at **different concentrations**  
- A **genetic perturbation** (e.g., CRISPR knockout, RNAi, overexpression)  

**Replicate**  
Repeated experimental samples under identical conditions (same perturbation, same concentration).  
Used to assess reproducibility and calculate metrics such as **replicate correlation** or **GRIT**.

**Channel**  
A fluorescence imaging band capturing a specific cellular component (e.g., nucleus, mitochondria, actin).  
Each channel provides complementary morphological information.

**Feature Aggregation**  
The process of combining single-cell features into summary statistics (e.g., mean, median) at the well or treatment level using tools like **PyCytominer**.

**Normalization**  
A data processing step to remove technical variability between wells or plates (e.g., z-score normalization, median-mAD scaling).

**Batch Effect**  
Non-biological differences introduced by technical variations between experiments or imaging runs.  
Must be corrected if necessaryto ensure meaningful biological comparisons.


In [1]:
import pandas as pd
from pycytominer import annotate, normalize, feature_select, consensus

# Load data

## Load CellProfiler plates data

In [2]:
data_list = []
plates = ["BR00127145", "BR00127146", "BR00127147", "BR00127148", "BR00127149"]
for plate in plates:
    data_tmp = pd.read_parquet('../00_input/' + plate + '.parquet')
    print(plate, "has shape", data_tmp.shape)  
    data_list.append(data_tmp)
df_cellprofiler = pd.concat(data_list)
df_cellprofiler = df_cellprofiler.reset_index(drop = True)
print("Aggregated Data has shape ", df_cellprofiler.shape)

BR00127145 has shape (384, 4765)
BR00127146 has shape (384, 4765)
BR00127147 has shape (382, 4765)
BR00127148 has shape (384, 4765)
BR00127149 has shape (384, 4765)
Aggregated Data has shape  (1918, 4765)


In [3]:
df_cellprofiler.head()

,Metadata_Source,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,source_4,BR00127145,A01,4254.899902,8091.399902,572.119995,583.989990,482.950012,495.049988,527.059998,...,92.257004,89.775002,85.719002,86.375999,85.484001,86.625000,87.264000,89.302002,86.958000,89.915001
1,source_4,BR00127145,A02,4784.100098,8854.400391,589.049988,608.909973,498.410004,514.130005,543.289978,...,96.736000,97.602997,92.363998,92.103996,92.021004,92.017998,92.512001,94.092003,92.139999,93.738998
2,source_4,BR00127145,A03,4107.000000,7763.700195,582.250000,564.059998,494.739990,477.140015,537.960022,...,99.619003,99.348000,93.452003,94.043999,93.553001,93.907997,94.482002,97.015999,94.483002,96.976997
3,source_4,BR00127145,A04,4183.600098,7986.299805,599.619995,572.669983,510.799988,485.290009,554.849976,...,109.550003,109.330002,102.180000,102.339996,102.010002,102.440002,102.830002,105.480003,102.970001,105.519997
4,source_4,BR00127145,A05,4222.500000,7984.399902,568.159973,554.690002,479.619995,465.660004,523.640015,...,113.559998,110.690002,104.790001,105.480003,104.580002,105.410004,106.290001,109.459999,106.320000,109.330002


In [4]:
df_cellprofiler.shape

(1918, 4765)

## Load JUMP compounds annotations 

In [5]:
df_compounds_infos = pd.read_csv("../00_input/compound.csv")

In [6]:
df_compounds_infos.head()

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_SMILES
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCc1nccn1-c1cccc(C2CCCN2C(=O)c2ccc(OCCN(C)C)cc...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,O=C1NCCCN1Cc1ccc(Cl)cc1
2,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCn1nccc1S(=O)(=O)N1CC2CCC1CNC2
3,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCNC(=O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...
4,JCP2022_000006,AAAROXVLYNJINN-UHFFFAOYSA-N,"InChI=1S/C16H20N6O/c1-16(2,3)22-13(10-5-6-10)7...",Cn1cc(-c2noc(-c3cc(C4CC4)n(C(C)(C)C)n3)n2)cn1


In [7]:
df_compounds_infos.shape

(115796, 4)

## Load compounds wells links

In [8]:
df_compounds_wells_links = pd.read_csv('../00_input/source4_batch13_target2.csv')

In [9]:
df_compounds_wells_links.head()

,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_JCP2022
0,source_4,BR00127145,A01,JCP2022_043547
1,source_4,BR00127145,A02,JCP2022_050797
2,source_4,BR00127145,A03,JCP2022_050997
3,source_4,BR00127145,A04,JCP2022_108326
4,source_4,BR00127145,A05,JCP2022_033924


In [10]:
df_compounds_wells_links.shape

(1918, 4)

## Load compounds annotations

In [11]:
df_compounds_annotations = pd.read_csv("../00_input/perturbation_control.csv")

In [12]:
df_compounds_annotations.head()

,Metadata_JCP2022,Metadata_pert_type,Metadata_Name,Metadata_modality
0,JCP2022_033924,negcon,DMSO,compound
1,JCP2022_037716,poscon,AMG900,compound
2,JCP2022_025848,poscon,dexamethasone,compound
3,JCP2022_046054,poscon,FK-866,compound
4,JCP2022_035095,poscon,LY2109761,compound


In [13]:
df_compounds_annotations.shape

(9, 4)

## Merge data

In [14]:
# merge df_cellprofiler and df_compounds_wells_links based on common columns Metadata_Source Metadata_Plate Metadata_Well
df_cellprofiler_compounds = pd.merge(
    df_cellprofiler,
    df_compounds_wells_links,
    on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"],
    how="inner"   # or "left", "right", "outer" depending on what you need
)

In [15]:
df_cellprofiler_compounds.shape

(1918, 4766)

In [16]:
# merge df_cellprofiler_compounds and df_compounds_wells_links based on common column Metadata_JCP2022
df_cellprofiler_compounds_infos = pd.merge(
    df_cellprofiler_compounds,
    df_compounds_infos,
    on=["Metadata_JCP2022"],
    how="inner"
)

In [17]:
df_cellprofiler_compounds_infos.shape

(1918, 4769)

In [18]:
# merge df_cellprofiler_compounds_infos and df_compounds_annotations based on common column Metadata_JCP2022
df_cellprofiler_compounds_infos_annotate = pd.merge(
    df_cellprofiler_compounds_infos,
    df_compounds_annotations,
    on=["Metadata_JCP2022"],
    how="left"
)

In [19]:
df_cellprofiler_compounds_infos_annotate.shape

(1918, 4772)

In [20]:
# df od levele 3
df_level3 = df_cellprofiler_compounds_infos_annotate

In [21]:
df_level3.shape

(1918, 4772)

## Save

In [22]:
df_level3.to_csv('../02_processed_data/df_level3.csv', index=False)

# Process data

## Aggregate (level 2 --> level 3)
In this JUMP example the data are already aggregated at the well level (level 3).
If in CellPainting dataset with profiles at the single cell level (level 2), you should aggregated the profiles with aggregate function of pycitominer

## Normalize (level 4a)

In [23]:
# Normalize by plates (and batch if exists)
df_level4a_list = []
for plate in plates:
    df_level4a_plate = normalize(
        profiles=df_level3[df_level3["Metadata_Plate"] == plate],
        features="infer", # If “infer”, then assume features are from CellProfiler output and prefixed with “Cells”, “Nuclei”, or “Cytoplasm”. 
        meta_features="infer", # If “infer”, then assume CellProfiler metadata features, identified by column names that begin with the Metadata_ prefix.”
        samples="Metadata_Name == 'DMSO'",
        method="mad_robustize",
    )
    df_level4a_list.append(df_level4a_plate)
    print("Plate", plate, "normalized")

df_level4a = pd.concat(df_level4a_list).reset_index(drop=True)
print("Normalized Data has shape:", df_level4a.shape)

Plate BR00127145 normalized
Plate BR00127146 normalized
Plate BR00127147 normalized
Plate BR00127148 normalized
Plate BR00127149 normalized
Normalized Data has shape: (1918, 3683)


In [24]:
df_level4a.to_csv('../02_processed_data/df_level4a.csv', index=False)

In [25]:
df_level4a.shape

(1918, 3683)

# Feature selection (level 4b)
Operations : list of feature selection operations 
- variance_threshold : removes features that have a variance under the threshold
- correlation_threshold removes redundant features
- drop_na_columns removes features with NaN values
- blocklist removes features that are a part of the feature blocklist

Default values :
- na_cutoff (float, default 0.05) – Proportion of missing values in a column to tolerate before removing.

corr_threshold (float, default 0.9) – Value between (0, 1) to exclude features above if any two features are correlated above this threshold.

corr_method (str, default "pearson") – Correlation type to compute. Allowed methods are “spearman”, “kendall” and “pearson”.

freq_cut (float, default 0.05) – Ratio (2nd most common feature val / most common). Must range between 0 and 1. Remove features lower than freq_cut. A low freq_cut will remove features that have large difference between the most common feature and second most common feature. (e.g. this will remove a feature: [1, 1, 1, 1, 0.01, 0.01, …])

unique_cut (float, default 0.01) – Ratio (num unique features / num samples). Must range between 0 and 1. Remove features less than unique cut. A low unique_cut will remove features that have very few different measurements compared to the number of samples.

blocklist_file (str, optional) – File location of datafrmame with with features to exclude. Note that if “blocklist” in operation then will remove standard blocklist default : https://github.com/cytomining/pycytominer/blob/main/pycytominer/data/blocklist_features.txt

In [26]:
feature_select_opts = [
    "variance_threshold",
    "drop_na_columns",
    "correlation_threshold",
    "blocklist",
    "drop_outliers",
]
df_level4b = feature_select(
    profiles=df_level4a, features="infer", samples="all", operation=feature_select_opts
)

In [27]:
print('There were :', len(df_level4a.columns)-len(df_level4b.columns),"features removed")
all_selected_features = [c for c in df_level4b.columns if not c.startswith("Metadata_")]
print('There are :', len(all_selected_features), 'selected features')

There were : 3118 features removed
There are : 555 selected features


In [28]:
df_level4b.to_csv('../02_processed_data/df_level4b.csv', index=False)

In [29]:
df_level4b.shape

(1918, 565)

# Consensus signature (level 5)
Group profiles per perturbation. Here we aggregated by chemicals but you can aggregate by chemical and/or concentration or other perturbation you are interested in.

In [30]:
df_level5 = consensus(
    profiles=df_level4b,
    replicate_columns=["Metadata_JCP2022"],  # replicate identifier, add concentration column if you have mutliple concentrtion
    features="infer",
    operation="modz",
)

/Users/ollitrault/Documents/projects/2025_CellPainting_Training/.env/lib/python3.11/site-packages/pycytominer/cyto_utils/modz.py:154: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [31]:
# merge df_level5 and df_compounds_annotations based on common column Metadata_JCP2022
df_level5 = pd.merge(
    df_level5,
    df_compounds_annotations,
    on=["Metadata_JCP2022"],
    how="left"   # or "left", "right", "outer" depending on what you need
)

In [32]:
df_level5.to_csv('../02_processed_data/df_level5.csv', index=False)

In [33]:
df_level5.head()

,Metadata_JCP2022,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_MaximumRadius,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,...,Nuclei_Texture_InfoMeas1_DNA_10_01_256,Nuclei_Texture_InfoMeas2_AGP_3_02_256,Nuclei_Texture_InfoMeas2_DNA_10_03_256,Nuclei_Texture_InverseDifferenceMoment_AGP_10_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_10_03_256,Nuclei_Texture_InverseDifferenceMoment_RNA_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256,Metadata_pert_type,Metadata_Name,Metadata_modality
0,JCP2022_000794,0.881888,-3.283020,-5.181463,-0.693973,-1.538441,-4.742888,1.114708,1.779054,-1.181462,...,-4.476324,-1.288912,3.322618,0.379173,-2.815993,0.647826,-0.574508,NaN,NaN,NaN
1,JCP2022_001036,0.676950,-0.193252,1.621191,-0.789116,-0.268256,-0.833730,-0.409720,0.048937,0.154339,...,-0.543275,-3.099220,0.655132,1.698192,-0.629725,1.393161,-0.980623,NaN,NaN,NaN
2,JCP2022_001275,-0.268107,0.407780,0.056508,-0.255188,0.025191,1.268643,-1.081036,-0.082914,0.750297,...,0.606763,-0.022665,-0.525418,0.728233,0.341216,0.804431,-0.738984,NaN,NaN,NaN
3,JCP2022_001890,0.738979,-5.782485,2.768373,0.029348,-0.280260,-9.427014,-1.712416,1.771081,-2.881955,...,-1.330497,2.271187,0.268864,-2.331185,-0.221517,-1.226987,3.074401,NaN,NaN,NaN
4,JCP2022_002118,0.426561,-2.923847,6.631440,0.197447,-2.067067,-4.106314,-0.575022,0.601895,-1.204755,...,-1.198537,-0.878925,1.406968,0.238938,-1.256263,0.245416,-0.215016,NaN,NaN,NaN


In [34]:
df_level5.shape

(302, 559)